Hàm trích xuất đặc trưng từ url

In [2]:
import torch
from urllib.parse import urlparse
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin


def extract_features(url):
    try:
        parsed_url = urlparse(url)
        
        # TLD
        tld = parsed_url.netloc.split('.')[-1]
        
        # Feature hashing cho TLD
        tld_hashed = feature_hash(tld)
        
        # URL Length
        url_length = len(url)
        
        # Is Domain IP
        is_domain_ip = 1 if parsed_url.netloc.replace('.', '').isdigit() else 0
        
        # Number of Subdomains
        subdomains = parsed_url.netloc.split('.')
        num_subdomains = len(subdomains)-2
        
        # Number of Obfuscated Characters
        num_obfuscated_chars = sum([url.count(char) for char in ['@', '-', '_', '~']])
        
        # Is HTTPS
        is_https = 1 if parsed_url.scheme == 'https' else 0
        
        # Number of Digits in URL
        num_digits = sum(c.isdigit() for c in url)
        
        # Number of Equals in URL
        num_equals = url.count('=')
        
        # Number of Question Marks in URL
        num_qmark = url.count('?')
        
        # Number of Ampersands in URL
        num_ampersand = url.count('&')
    
        return {
            'TLD': tld_hashed,
            'URLLength': url_length,
            'IsDomainIP': is_domain_ip,
            'NoOfSubDomain': num_subdomains,
            'NoOfObfuscatedChar': num_obfuscated_chars,
            'IsHTTPS': is_https,
            'NoOfDegitsInURL': num_digits,
            'NoOfEqualsInURL': num_equals,
            'NoOfQMarkInURL': num_qmark,
            'NoOfAmpersandInURL': num_ampersand
        }
    except:
        print("Lỗi khi trichs xuất đặc trưng")

def feature_hash(value, num_features=100):
    # Sử dụng hàm hash để ánh xạ giá trị vào một phạm vi num_features
    return hash(value) % num_features
# Example URL to test
#test_url = "mailto://www.arara.org"
test_url = "https://www.ci.boerne.tx.us"

# Extract features from the test URL
print(extract_features(test_url))



{'TLD': 46, 'URLLength': 27, 'IsDomainIP': 0, 'NoOfSubDomain': 3, 'NoOfObfuscatedChar': 0, 'IsHTTPS': 1, 'NoOfDegitsInURL': 0, 'NoOfEqualsInURL': 0, 'NoOfQMarkInURL': 0, 'NoOfAmpersandInURL': 0}


Hàm trích xuất hyperlinks từ url, giới hạn max = 20

In [3]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from requests.exceptions import ConnectionError

def extract_hyperlinks(url):
    try:
        # Sử dụng requests để lấy nội dung của trang web
        response = requests.get(url)
        
        # Kiểm tra xem request có thành công không
        if response.status_code != 200:
            return []  # Trả về một danh sách rỗng nếu request không thành công
        
        # Sử dụng BeautifulSoup để phân tích cú pháp HTML
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Tìm tất cả các thẻ <a> trong trang web
        links = []
        max_links = 20
        count = 0
        for link in soup.find_all('a'):
            href = link.get('href')
            if href:  # Đảm bảo thuộc tính href tồn tại
                # Đưa đường dẫn tương đối thành đường dẫn tuyệt đối
                absolute_url = urljoin(url, href)
                # Thêm liên kết vào danh sách nếu nó không trùng lặp
                if absolute_url not in links and count < max_links:
                    links.append(absolute_url)
                    count += 1
                elif count >= max_links:
                    break  # Dừng vòng lặp nếu số lượng link vượt quá 20
        
        return links
    except:
        print("lỗi khi trích xuất hyperlinks", end = ',')
extract_hyperlinks ('http://www.expoglobalservice.com')

[]

Hàm biểu diễn mỗi url thành 1 đồ thị

In [4]:
import torch
from torch_geometric.data import Data

def url_to_graph(url):
    try:
    
        features = extract_features(url)
        
        # Tạo danh sách các nút và các đặc trưng tương ứng
        nodes = [url]
        node_features = [list(features.values())]
        
        # Trích xuất liên kết từ URL và thêm vào danh sách các nút và đặc trưng
        hyperlinks = extract_hyperlinks(url)
        if len(hyperlinks) != 0: 
            for link in hyperlinks:
                nodes.append(link)
                features_link = extract_features(link)
                node_features.append(list(features_link.values()))
        
        # Tạo tensor chứa các đặc trưng của các nút
        x = torch.tensor(node_features, dtype=torch.float)

       # import torch

        # Giả sử x là tensor ban đầu
        # x = ... (tensor có kích thước khác mong muốn)

        # Kích thước mong muốn
        desired_size = [21, 10]

        # Nếu tensor x có kích thước lớn hơn kích thước mong muốn, bạn có thể cắt bớt hàng
        if x.size(0) > desired_size[0]:
            x = x[:desired_size[0], :]

        # Nếu tensor x có kích thước nhỏ hơn kích thước mong muốn, bạn có thể thêm hàng padding
        elif x.size(0) < desired_size[0]:
            # Tạo tensor padding với giá trị 0 có kích thước phù hợp
            padding_rows = desired_size[0] - x.size(0)
            padding = torch.zeros(padding_rows, x.size(1))
            # Thêm tensor padding vào cuối tensor x
            x = torch.cat((x, padding), dim=0)

        # Kiểm tra kích thước mới của tensor x
        print(x.size())  # Kết quả: torch.Size([21, 10])

        
        # Chuyển đổi cạnh thành danh sách cạnh (dạng (src, dst))
        edge_index = []
        
        # Thêm các cạnh từ URL gốc đến các liên kết mới
        url_index = nodes.index(url)
        if len(hyperlinks) !=0:
            for i, link in enumerate(hyperlinks):
                link_index = nodes.index(link)
                edge_index.append([url_index, link_index])

        if len(edge_index) !=0:
            edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()


        # Desired size (2, 20)
            new_size = (2, 20)

        # Initialize a tensor filled with -1 as padding value
            padded_edge_index = torch.full(new_size, -1, dtype=torch.long)

        # Copy data from edge_index to padded_edge_index
            padded_edge_index[:, :edge_index.size(1)] = edge_index

        # padded_edge_index is now padded to the size (2, 20)
            edge_index = padded_edge_index

            print(edge_index.size())
        else:
            edge_index = torch.zeros((2, 20), dtype=torch.long)

        # Tạo đối tượng Data từ các đặc trưng và cạnh
        data = Data(x=x, edge_index=edge_index)

        return data
    except:
        print("Lỗi khi url to graph")
        print(url)

# Example URL
url = "https://www.ci.boerne.tx.us"

# Convert URL to Graph
graph_data = url_to_graph(url)
print(graph_data.x)
print(graph_data.edge_index)



/Users/rollie/anaconda3/envs/myenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([21, 10])
torch.Size([2, 20])
tensor([[46., 27.,  0.,  3.,  0.,  1.,  0.,  0.,  0.,  0.],
        [46., 66.,  0.,  3.,  4.,  1., 20.,  0.,  0.,  0.],
        [46., 28.,  0.,  3.,  0.,  1.,  0.,  0.,  0.,  0.],
        [46., 42.,  0.,  3.,  0.,  1.,  0.,  0.,  0.,  0.],
        [46., 41.,  0.,  3.,  0.,  1.,  2.,  0.,  0.,  0.],
        [46., 39.,  0.,  3.,  0.,  1.,  1.,  0.,  0.,  0.],
        [46., 46.,  0.,  3.,  1.,  1.,  3.,  0.,  0.,  0.],
        [46., 42.,  0.,  3.,  0.,  1.,  2.,  0.,  0.,  0.],
        [46., 41.,  0.,  3.,  2.,  1.,  3.,  0.,  0.,  0.],
        [46., 41.,  0.,  3.,  0.,  1.,  3.,  0.,  0.,  0.],
        [29., 43.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.],
        [46., 53.,  0.,  3.,  1.,  1.,  4.,  0.,  0.,  0.],
        [46., 42.,  0.,  3.,  0.,  1.,  4.,  0.,  0.,  0.],
        [29., 41.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.],
        [46., 47.,  0.,  3.,  1.,  1.,  3.,  0.,  0.,  0.],
        [46., 46.,  0.,  3.,  1.,  1.,  4.,  0.,  0.,  0.],

tạo tập train_loader và test_loader, chuẩn bị cho train mô hình GNN

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
from torch_scatter import scatter_max
from torch_geometric.nn import GCNConv, global_max_pool
from torch_geometric.data import Data


# Hàm extract_features và extract_hyperlinks đã được định nghĩa trước đó
def global_max_pool(x, batch):
    # Tính max pooling trên các nút trong từng batch
    return scatter_max(x, batch, dim=0)[0]

# Hàm tạo đồ thị từ URL
def url_to_graph_data(url, label):
    
    print(url)
    try:
        # Biểu diễn URL thành đồ thị
        graph_data = url_to_graph(url)
        
        


        # Tạo dữ liệu PyTorch Geometric từ đồ thị và nhãn
        data = Data(x=graph_data.x, edge_index=graph_data.edge_index, y=torch.tensor(label, dtype=torch.long))

        
        return data
    except:
        print("Lỗi khi url to graph")

test_url="https://www.ci.boerne.tx.us"
graph_data=url_to_graph_data(test_url,1)
print(graph_data.x)
print(graph_data.edge_index)
print(graph_data.y)






https://www.ci.boerne.tx.us
torch.Size([21, 10])
torch.Size([2, 20])
tensor([[46., 27.,  0.,  3.,  0.,  1.,  0.,  0.,  0.,  0.],
        [46., 66.,  0.,  3.,  4.,  1., 20.,  0.,  0.,  0.],
        [46., 28.,  0.,  3.,  0.,  1.,  0.,  0.,  0.,  0.],
        [46., 42.,  0.,  3.,  0.,  1.,  0.,  0.,  0.,  0.],
        [46., 41.,  0.,  3.,  0.,  1.,  2.,  0.,  0.,  0.],
        [46., 39.,  0.,  3.,  0.,  1.,  1.,  0.,  0.,  0.],
        [46., 46.,  0.,  3.,  1.,  1.,  3.,  0.,  0.,  0.],
        [46., 42.,  0.,  3.,  0.,  1.,  2.,  0.,  0.,  0.],
        [46., 41.,  0.,  3.,  2.,  1.,  3.,  0.,  0.,  0.],
        [46., 41.,  0.,  3.,  0.,  1.,  3.,  0.,  0.,  0.],
        [29., 43.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.],
        [46., 53.,  0.,  3.,  1.,  1.,  4.,  0.,  0.,  0.],
        [46., 42.,  0.,  3.,  0.,  1.,  4.,  0.,  0.,  0.],
        [29., 41.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.],
        [46., 47.,  0.,  3.,  1.,  1.,  3.,  0.,  0.,  0.],
        [46., 46.,  0.,  3.,  1

In [ ]:
# Load dataset từ file hoặc database
# Assume dataset là một danh sách các tuples (url, label)
df = pd.read_csv('/Users/rollie/Documents/URL_Phishing_Detection/notebooks/100gnn.csv')
newdf = list(zip(df['URL'], df['label']))


# Chia dataset thành tập huấn luyện và tập kiểm tra
train_data, test_data = train_test_split(newdf, test_size=0.2, random_state=42)

print("Biểu diễn dữ liệu cho tập HL")
# Biểu diễn mỗi URL trong tập huấn luyện thành đồ thị và tạo DataLoader cho tập huấn luyện
#train_graph_data = [url_to_graph_data(URL, label) for URL, label in train_data]
train_graph_data = []
for URL, label in train_data:
    try:
        graph_data = url_to_graph_data(URL, label)
        train_graph_data.append(graph_data)
    except Exception as e:
        print(f"Error processing URL: {URL}. Error: {e}")

train_loader = DataLoader(train_graph_data, batch_size=8, shuffle=True)

print("Biểu diễn dữ liệu cho tập kiểm tra")
# Biểu diễn mỗi URL trong tập kiểm tra thành đồ thị và tạo DataLoader cho tập kiểm tra
test_graph_data = []
for URL, label in test_data:
    try:
        graph_data = url_to_graph_data(URL, label)
        test_graph_data.append(graph_data)
    except Exception as e:
        print(f"Error processing URL: {URL}. Error: {e}")
test_loader = DataLoader(test_graph_data, batch_size=2, shuffle=False)

In [19]:
print(len(train_graph_data))
print(len(test_graph_data))

80
20


In [8]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader


# Load dataset từ file hoặc database
# Assume dataset là một danh sách các tuples (url, label)
df = pd.read_csv('/Users/rollie/Documents/URL_Phishing_Detection/notebooks/gnn-file-cut.csv')
newdf = list(zip(df['URL'], df['label']))


# Chia dataset thành tập huấn luyện và tập kiểm tra
train_data, test_data = train_test_split(newdf, test_size=0.2, random_state=42)

print("Biểu diễn dữ liệu cho tập HL")
# Biểu diễn mỗi URL trong tập huấn luyện thành đồ thị và tạo DataLoader cho tập huấn luyện
#train_graph_data = [url_to_graph_data(URL, label) for URL, label in train_data]
train_graph_data = []
for URL, label in train_data:
    try:
        graph_data = url_to_graph_data(URL, label)
        train_graph_data.append(graph_data)
    except Exception as e:
        print(f"Error processing URL: {URL}. Error: {e}")

train_loader = DataLoader(train_graph_data, batch_size=2, shuffle=True)

print("Biểu diễn dữ liệu cho tập kiểm tra")
# Biểu diễn mỗi URL trong tập kiểm tra thành đồ thị và tạo DataLoader cho tập kiểm tra
test_graph_data = []
for URL, label in test_data:
    try:
        graph_data = url_to_graph_data(URL, label)
        test_graph_data.append(graph_data)
    except Exception as e:
        print(f"Error processing URL: {URL}. Error: {e}")
test_loader = DataLoader(test_graph_data, batch_size=2, shuffle=False)
# Định nghĩa mô hình GNN
class GNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GNNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = global_max_pool(x, data.batch)  # Global max pooling
        x = self.fc(x)
        return x
# Khởi tạo mô hình
model = GNNModel(input_dim=10, hidden_dim=10, output_dim=2)

# Định nghĩa hàm loss và optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Chia dữ liệu thành 5 fold
kf = KFold(n_splits=5)
# Đánh giá trên từng fold
fold_accuracies = []
for train_index, test_index in kf.split(train_graph_data):  # dataset là tập dữ liệu đầy đủ
    train_data = [train_graph_data[i] for i in train_index]
    print(len(train_data))
    print("...")
    
    test_data = [train_graph_data[i] for i in test_index]
    print(len(test_data))
    from torch_geometric.data import Data



# Tạo DataLoader với custom collate function
    #train_loader = DataLoader(train_data, batch_size=8, shuffle=True, collate_fn=custom_collate)
    #test_loader = DataLoader(test_data, batch_size=2, shuffle=False, collate_fn=custom_collate)

    train_loader = DataLoader(train_data, batch_size= 8, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=2, shuffle=False)


 # Huấn luyện mô hình
    for epoch in range(10):  # Ví dụ cho 10 epochs
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            output = model(batch)
            loss = criterion(output, batch.y)
            loss.backward()
            optimizer.step()

    # Đánh giá mô hình trên tập kiểm tra
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for batch in test_loader:
            output = model(batch)
            _, predicted = torch.max(output.data, 1)
            total += batch.y.size(0)
            correct += (predicted == batch.y).sum().item()

        fold_accuracy = correct / total
        fold_accuracies.append(fold_accuracy)

# Tính trung bình độ chính xác trên các fold
mean_accuracy = sum(fold_accuracies) / len(fold_accuracies)
print('Mean accuracy across 5 folds:', mean_accuracy)


Biểu diễn dữ liệu cho tập HL
https://www.coppercanyontrails.org
torch.Size([21, 10])
torch.Size([2, 20])
https://www.stayarlington.com
torch.Size([21, 10])
torch.Size([2, 20])
https://www.ssmt-reviews.com
torch.Size([21, 10])
torch.Size([2, 20])
https://www.manilalivewire.com
torch.Size([21, 10])
torch.Size([2, 20])
https://www.questpaymentsystems.com
torch.Size([21, 10])
torch.Size([2, 20])
https://www.goriverwalk.com
torch.Size([21, 10])
torch.Size([2, 20])
http://www.expoglobalservice.com
torch.Size([21, 10])
https://www.ridiculousupside.com
torch.Size([21, 10])
torch.Size([2, 20])
https://www.allnews.ch
torch.Size([21, 10])
torch.Size([2, 20])
https://www.hajker.hu
torch.Size([21, 10])
torch.Size([2, 20])
https://www.bwpinebluffar.com
torch.Size([21, 10])
torch.Size([2, 20])
https://www.taptapkaboom.com
torch.Size([21, 10])
torch.Size([2, 20])
https://www.rowmaninternational.com
torch.Size([21, 10])
torch.Size([2, 20])
https://www.yumingschool.org
torch.Size([21, 10])
torch.Size([2

ValueError: 'GCNConv' received a tuple of node features as input while this layer does not support bipartite message passing. Please try other layers such as 'SAGEConv' or 'GraphConv' instead